In [ ]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import pandas as pd

plt.rcParams.update({
    "figure.dpi": 120,
    "axes.prop_cycle": plt.cycler(color=["#1f77b4", "#d62728", "#2ca02c", "#ff7f0e"]),
})

In [ ]:
try:
    NOTEBOOK_DIR = Path(__file__).resolve().parent
except NameError:
    NOTEBOOK_DIR = Path.cwd()

ARTIFACT_DIR = NOTEBOOK_DIR / "artifacts" / "precompute_data"
RUN_METADATA_CSV = ARTIFACT_DIR / "precompute_run_metadata.csv"

print(f"Artifacts dir: {ARTIFACT_DIR}")
print(f"Metadata CSV: {RUN_METADATA_CSV}")

def load_run_metadata(order_by: tuple[str, ...] | None = ("lattice", "resolution", "mesh_size")) -> pd.DataFrame:
    if not RUN_METADATA_CSV.exists():
        raise FileNotFoundError("Run the precompute notebook first to materialize run metadata.")
    df = pd.read_csv(RUN_METADATA_CSV)
    if "output_path" in df.columns:
        df["output_path"] = df["output_path"].astype(str)
    if order_by:
        df = df.sort_values(list(order_by)).reset_index(drop=True)
    return df

In [ ]:
DIVERGING_CMAP = colors.LinearSegmentedColormap.from_list(
    "blue_white_red",
    ["#0b4f8a", "#ffffff", "#b40426"],
)
CLAMP_CMAP = colors.ListedColormap(["#1f77b4", "#d62728"])

def _pivot_matrix(df: pd.DataFrame, value_col: str) -> np.ndarray:
    pivot = df.pivot(index="iy", columns="ix", values=value_col).sort_index(ascending=False)
    return pivot.values

def _symlog_norm(arr: np.ndarray, ratio: float = 1e-2) -> colors.SymLogNorm:
    finite = np.abs(arr[np.isfinite(arr)])
    if finite.size == 0:
        return colors.SymLogNorm(linthresh=1e-9, linscale=1.0, vmin=-1.0, vmax=1.0)
    max_abs = float(finite.max())
    if max_abs == 0.0:
        max_abs = 1.0
    linthresh = max(max_abs * ratio, 1e-9)
    return colors.SymLogNorm(linthresh=linthresh, linscale=1.0, vmin=-max_abs, vmax=max_abs)

def _load_epsilon_fourier_maps(csv_path: Path) -> dict[str, np.ndarray]:
    df = pd.read_csv(csv_path)
    for col in ("real", "imag"):
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df["magnitude"] = np.sqrt(df["real"].fillna(0.0) ** 2 + df["imag"].fillna(0.0) ** 2)
    return {
        "magnitude": _pivot_matrix(df, "magnitude"),
        "real": _pivot_matrix(df, "real"),
        "imag": _pivot_matrix(df, "imag"),
    }

def _load_fft_workspace_maps(csv_path: Path) -> dict[str, np.ndarray]:
    df = pd.read_csv(csv_path)
    df["k_plus_g_sq"] = pd.to_numeric(df["k_plus_g_sq"], errors="coerce")
    df["clamped"] = df["clamped"].astype(int)
    return {
        "k_plus_g_sq": _pivot_matrix(df, "k_plus_g_sq"),
        "clamped": _pivot_matrix(df, "clamped"),
    }

def _resolve_artifacts(row: pd.Series) -> tuple[dict[str, Any], dict[str, np.ndarray], dict[str, np.ndarray]]:
    json_path = Path(row["output_path"])
    payload = json.loads(json_path.read_text())
    files = payload.get("files", {})
    eps_csv = files.get("epsilon_fourier_csv")
    fft_csv = files.get("fft_workspace_csv")
    if not eps_csv or not fft_csv:
        raise FileNotFoundError(f"Missing CSV references for {row['tag']}")
    eps_maps = _load_epsilon_fourier_maps(json_path.with_name(eps_csv))
    fft_maps = _load_fft_workspace_maps(json_path.with_name(fft_csv))
    return payload, eps_maps, fft_maps

In [ ]:
def plot_configuration(row: pd.Series, ratio: float = 1e-2) -> None:
    payload, eps_maps, fft_maps = _resolve_artifacts(row)
    te_eff = payload.get("te_eps_eff", row.get("te_eps_eff"))
    tm_eff = payload.get("tm_eps_eff", row.get("tm_eps_eff"))
    if te_eff is not None and tm_eff is not None and pd.notna(te_eff) and pd.notna(tm_eff):
        title_suffix = f"TE eff={te_eff:.3f} · TM eff={tm_eff:.3f}"
    else:
        title_suffix = "effective eps unavailable"
    real_norm = _symlog_norm(eps_maps["real"], ratio)
    imag_norm = _symlog_norm(eps_maps["imag"], ratio)
    fig = plt.figure(figsize=(14, 8))
    gs = fig.add_gridspec(2, 3, height_ratios=[1, 1], wspace=0.25, hspace=0.3)
    ax_mag = fig.add_subplot(gs[0, 0])
    ax_real = fig.add_subplot(gs[0, 1])
    ax_imag = fig.add_subplot(gs[0, 2])
    ax_k = fig.add_subplot(gs[1, 0:2])
    ax_clamp = fig.add_subplot(gs[1, 2])
    im_mag = ax_mag.imshow(eps_maps["magnitude"], origin="lower", cmap="viridis")
    fig.colorbar(im_mag, ax=ax_mag, fraction=0.046, pad=0.02, label="|ε(G)|")
    im_real = ax_real.imshow(eps_maps["real"], origin="lower", cmap=DIVERGING_CMAP, norm=real_norm)
    fig.colorbar(im_real, ax=ax_real, fraction=0.046, pad=0.02, label="Re ε(G)")
    im_imag = ax_imag.imshow(eps_maps["imag"], origin="lower", cmap=DIVERGING_CMAP, norm=imag_norm)
    fig.colorbar(im_imag, ax=ax_imag, fraction=0.046, pad=0.02, label="Im ε(G)")
    im_k = ax_k.imshow(fft_maps["k_plus_g_sq"], origin="lower", cmap="magma")
    fig.colorbar(im_k, ax=ax_k, fraction=0.046, pad=0.02, label="|k+G|²")
    im_clamp = ax_clamp.imshow(fft_maps["clamped"], origin="lower", cmap=CLAMP_CMAP, vmin=0, vmax=1)
    clamp_bar = fig.colorbar(im_clamp, ax=ax_clamp, ticks=[0, 1], fraction=0.046, pad=0.02, label="Clamp")
    clamp_bar.ax.set_yticklabels(["free", "clamped"])
    ax_mag.set_title('|ε(G)|')
    ax_real.set_title('Re ε(G) — zero is white')
    ax_imag.set_title('Im ε(G)')
    ax_k.set_title('|k+G|²')
    ax_clamp.set_title('Clamp mask')
    for ax in (ax_mag, ax_real, ax_imag, ax_k, ax_clamp):
        ax.set_xticks([])
        ax.set_yticks([])
    fig.suptitle(f"{row['tag']} — {title_suffix}", fontsize=14, y=0.98)
    plt.show()

In [ ]:
run_metadata_df = load_run_metadata()
run_metadata_df[["tag", "lattice", "resolution", "mesh_size", "output_path"]]

In [ ]:
def render_configurations(df: pd.DataFrame, limit: int | None = None) -> None:
    subset = df.head(limit) if limit else df
    for idx, row in subset.iterrows():
        print(f"Rendering {row['tag']} ({idx + 1}/{len(subset)})")
        plot_configuration(row)

# Plot every configuration generated by the precompute notebook
render_configurations(run_metadata_df)